In [2]:
import json 

# Load the emotions from the JSON file
with open('./data/emotions.json', 'r') as f:
    emotions = json.load(f)

#convert the emotions.positive to a list in a new variable
emotions_list = '\n'.join([str(elem)for i, elem in enumerate (emotions['positive'])])
emotions_list = emotions_list + '\n'.join([str(elem)for i, elem in enumerate (emotions['negative'])])
emotions_list = emotions_list + '\n'.join([str(elem)for i, elem in enumerate (emotions['complex'])])

print(emotions_list)

Joyful
Grateful
Serene
Interested
Hopeful
Proud
Amusement
Inspired
Awed
LovedAngry
Sad
Afraid
Disgusted
Ashamed
Guilty
Envious
Jealous
Frustrated
LonelyNostalgic
Empathetic
Sympathetic
Content
Satisfied
Confused
Anticipative
Relieved


In [3]:
# import langchain modules for ollama
from langchain_community.llms import Ollama

llm = Ollama(model="llama3:8b-instruct-q6_K")
llm.invoke("Are you listening to me?")


KeyboardInterrupt: 

In [ ]:
# import langchain prompttemplate
from langchain.prompts import PromptTemplate

# define the template
prompt_template = """
{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>
{{ .Prompt }}
You are PHD candidate in the psychology department. You have been given the task of labeling text from students to determine their emotional state.
You will be given a sentence or two from the student. You will attempt to infer the students emotional state from the text. 

You will limit your the emotions to the following list of emotions:
{emotion_list}

Your task is to provide a single word that best describes the emotional state from the first person perspective.
Sad
Happy
Angry

If you believe there could be multiple emotions, choose the one that is most prevalent. 
If the text refers to multiple people, you will only be given the emotion from the perspective of the first person. Choose the best emotion for the first person. 
There is not penalty for providing a wrong answer.

The students text:
{text}

Remember to only respond with a single word. Do not explain your reasoning. 1 word response only. Such as "Happy", "Sad" or "Angry"


SINGLE 1 WORD ANSWER ONLY.
<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt_formatted_str: str = prompt.format(
    emotion_list=emotions_list, text="i was feeling at the start didnt want to move much at all was really glad to experience this glimpse into the sort of vibrant energy i will gain through out the year"
)

llm = Ollama(model="llama3:8b-instruct-q6_K")
prediction = llm.predict(prompt_formatted_str)
print(prediction)


    

In [ ]:
# import langchain prompttemplate
from langchain.prompts import PromptTemplate

# define the template
prompt_template = """
As a psychology Ph.D. candidate, your task is to identify the predominant emotional state of a student based on their provided text. 
You should limit your responses to one of the following emotions: {emotion_list} Ensure your response reflects the emotional state 
from the first person's perspective. Respond with only one word that best captures the emotion. Do not provide explanations. 
Use just one of the specified emotions as your answer.

Students' text: {text}

Response: [Your one-word emotion here]
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt_formatted_str: str = prompt.format(
    emotion_list=emotions_list, text="i was feeling at the start didnt want to move much at all was really glad to experience this glimpse into the sort of vibrant energy i will gain through out the year"
)

llm = Ollama(model="llama3:8b-instruct-q6_K")
prediction = llm.predict(prompt_formatted_str)
print(prediction)


    

In [ ]:
# let's make the above into a function called predict_emotion
# update the function to take an optional model, default to mistral
# if no model is passed, use mistral


def predict_emotion(text, model="llama3:8b"):  
    prompt = PromptTemplate.from_template(prompt_template)
    prompt_formatted_str: str = prompt.format(
        emotion_list=emotions_list, text=text
    )

    llm = Ollama(model=model)
    prediction = llm.predict(prompt_formatted_str)
    return prediction


In [ ]:
!pip install pandas
!pip install colorama

In [ ]:
# read in the ./data/emotions_sentiment_sample.csv file using pandas and send the text column to the predict_emotion function
import pandas as pd
from colorama import Fore, Style
df = pd.read_csv('./data/emotions_sentiment_sample.csv')
#loop through the text column and call the predict_emotion function
for text in df['text']:
    prediction = predict_emotion(text, "llama3")
    #check to see if there is more than one word in prediction, if so, split and take the first word only 
    if len(prediction.split()) > 1:
        prediction = prediction.split()[0]
    
    #strip any whitespace from the prediction or punctuation from prediction
    prediction = prediction.strip().strip('.,')

    print(Fore.BLUE + prediction + Style.RESET_ALL + " : " + text )

